# An Introduction to Models in Pyro

The basic unit of probabilistic programs is the _stochastic function_. 
This is an arbitrary Python callable that combines two ingredients:

- deterministic Python code; and
- primitive stochastic functions that call a random number generator

Concretely, a stochastic function can be any Python object with a `__call__()` method, like a function, a method, or a PyTorch `nn.Module`.

Throughout the tutorials and documentation, we will often call stochastic functions *models*, since stochastic functions can be used to represent simplified or abstract descriptions of a process by which data are generated.  Expressing models as stochastic functions means that models can be composed, reused, imported, and serialized just like regular Python callables. 

In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

## Primitive Stochastic Functions

Primitive stochastic functions, or distributions, are an important class of stochastic functions for which we can explicitly compute the probability of the outputs given the inputs.  As of PyTorch 0.4 and Pyro 0.2, Pyro uses PyTorch's [distribution library](http://pytorch.org/docs/master/distributions.html). You can also create custom distributions using [transforms](http://pytorch.org/docs/master/distributions.html#module-torch.distributions.transforms).

Using primitive stochastic functions is easy. For example, to draw a sample `x` from the unit normal distribution $\mathcal{N}(0,1)$ we do the following:

In [17]:
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample((2,3)) # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)
print("log likelihood", normal.log_prob(x).prod(), torch.distributions.Normal(loc + 0.2, scale).log_prob(x).prod())

sample tensor([[-0.4594, -0.5810,  0.4576],
        [-0.5299, -0.7282,  1.2456]])
log prob tensor([[-1.0245, -1.0877, -1.0236],
        [-1.0593, -1.1840, -1.6946]])
log likelihood tensor(2.4245) tensor(3.1045)


In [30]:
torch.manual_seed(22)
normal1 = torch.distributions.Normal(loc, scale)
normal2 = torch.distributions.Normal(loc, scale)
print(normal1.rsample((10,)))
#print(normal2.rsample((10,)))

tensor([ 1.0302, -0.5073, -0.1017,  1.3587, -3.0745, -0.7659,  0.7240,  0.2416,
        -1.5601,  1.3164])


Here, `torch.distributions.Normal` is an instance of the `Distribution` class that takes parameters and provides sample and score methods. Pyro's distribution library `pyro.distributions` is a thin wrapper around `torch.distributions` because we want to make use of PyTorch's fast tensor math and autograd capabilities during inference.

## A Simple Model

All probabilistic programs are built up by composing primitive stochastic functions and deterministic computation. Since we're ultimately interested in probabilistic programming because we want to model things in the real world, let's start with a model of something concrete. 

Let's suppose we have a bunch of data with daily mean temperatures and cloud cover. We want to reason about how temperature interacts with whether it was sunny or cloudy. A simple stochastic function that describes how that data might have been generated is given by:

In [31]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

Let's go through this line-by-line. First, in lines 2 we define a binary random variable 'cloudy', which is given by a draw from the Bernoulli distribution with a parameter of `0.3`. Since the Bernoulli distributions return `0`s or `1`s, in line 3 we convert the value `cloudy` to a string so that return values of `weather` are easier to parse. So according to this model 30% of the time it's cloudy and 70% of the time it's sunny.

In lines 4-5 we define the parameters we're going to use to sample the temperature in lines 6. These parameters depend on the particular value of `cloudy` we sampled in line 2. For example, the mean temperature is 55 degrees (Fahrenheit) on cloudy days and 75 degrees on sunny days. Finally we return the two values `cloudy` and `temp` in line 7.

However, `weather` is entirely independent of Pyro - it only calls PyTorch. We need to turn it into a Pyro program if we want to use this model for anything other than sampling fake data.

## The `pyro.sample` Primitive

To turn `weather` into a Pyro program, we'll replace the `torch.distribution`s with `pyro.distribution`s and the `.sample()` and `.rsample()` calls with calls to `pyro.sample`, one of the core language primitives in Pyro. Using `pyro.sample` is as simple as calling a primitive stochastic function with one important difference:

In [43]:
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x, x.item())

tensor(-0.4513) -0.45131391286849976


Just like a direct call to `torch.distributions.Normal().rsample()`, this returns a sample from the unit normal distribution. The crucial difference is that this sample is _named_. Pyro's backend uses these names to uniquely identify sample statements and _change their behavior at runtime_ depending on how the enclosing stochastic function is being used. As we will see, this is how Pyro can implement the various manipulations that underlie inference algorithms.

Now that we've introduced `pyro.sample` and `pyro.distributions` we can rewrite our simple model as a Pyro program:

In [45]:
def weather():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

In [46]:
for _ in range(3):
    print(weather())

('cloudy', 62.996036529541016)
('sunny', 75.35819244384766)
('cloudy', 67.18570709228516)


Procedurally, `weather()` is still a non-deterministic Python callable that returns two random samples. Because the randomness is now invoked with `pyro.sample`, however, it is much more than that. In particular `weather()` specifies a joint probability distribution over two named random variables: `cloudy` and `temp`. As such, it defines a probabilistic model that we can reason about using the techniques of probability theory. For example we might ask: if I observe a temperature of 70 degrees, how likely is it to be cloudy? How to formulate and answer these kinds of questions will be the subject of the next tutorial.

## Universality: Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow

We've now seen how to define a simple model. Building off of it is easy. For example:

In [51]:
def ice_cream_sales():
    cloudy, temp = weather()
    print(cloudy, temp)
    mean_expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(mean_expected_sales, 10.0))
    return ice_cream

In [63]:
for _ in range(4):
    sales = ice_cream_sales()
    print(sales, type(sales))

sunny 63.248416900634766
tensor(49.8173) <class 'torch.Tensor'>
cloudy 58.883968353271484
tensor(36.2058) <class 'torch.Tensor'>
cloudy 65.47975158691406
tensor(40.9174) <class 'torch.Tensor'>
sunny 99.03781127929688
tensor(205.5299) <class 'torch.Tensor'>


This kind of modularity, familiar to any programmer, is obviously very powerful. But is it powerful enough to encompass all the different kinds of models we'd like to express?

It turns out that because Pyro is embedded in Python, stochastic functions can contain arbitrarily complex deterministic Python and randomness can freely affect control flow. For example, we can construct recursive functions that terminate their recursion nondeterministically, provided we take care to pass `pyro.sample` unique sample names whenever it's called. For example we can define a geometric distribution that counts the number of failures until the first success like so:

In [55]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

In [60]:
print(geometric(0.1))

8


Note that the names `x_0`, `x_1`, etc., in `geometric()` are generated dynamically and that different executions can have different numbers of named random variables. 

We are also free to define stochastic functions that accept as input or produce as output other stochastic functions:

In [66]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

In [77]:
fn = make_normal_normal()
print(fn(torch.linspace(1,10,10)), type(fn))

tensor([  1.9055,   1.8148,   0.9238, -22.5844,   1.9626,   5.3471,   2.5478,
          6.2963,  13.1204, -47.5867]) <class 'function'>


In [73]:
np = normal_product(0.1, 2.0)
np, type(np)

(tensor(-0.0483), torch.Tensor)

Here `make_normal_normal()` is a stochastic function that takes one argument and which, upon execution, generates three named random variables.

The fact that Pyro supports arbitrary Python code like this&mdash;iteration, recursion, higher-order functions, etc.&mdash;in conjuction with random control flow means that Pyro stochastic functions are _universal_, i.e. they can be used to represent any computable probability distribution. As we will see in subsequent tutorials, this is incredibly powerful. 

It is worth emphasizing that this is one reason why Pyro is built on top of PyTorch: dynamic computational graphs are an important ingredient in allowing for universal models that can benefit from GPU-accelerated tensor math.

## Next Steps

We've shown how we can use stochastic functions and primitive distributions to represent models in Pyro. In order to learn models from data and reason about them we need to be able to do inference. This is the subject of the [next tutorial](intro_part_ii.ipynb).